nouvelles specs on part d'un nouvel atlas de 10 régions et on voit quels sont les voxels qui sont en correlations avec ces régions  

In [1]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine

from nipype.interfaces.utility import Function, IdentityInterface

from nipype.interfaces.fsl.maths import MathsCommand
from nipype.interfaces.fsl.utils import PlotMotionParams   # to plot moco variabl
from nipype import SelectFiles, Node

In [29]:
# creation of a subworflow to process ROI Correlations
connectivity = pe.Workflow(name='connectivity')

In [30]:
arimadir = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/arimaResidus'
atlasdir = '/scratch/user/hirsch/datadir/data_set/t0009/repos01/Atlases'


from nipype import SelectFiles, Node
templates = dict(arimaFile=arimadir+ "/" + "*.nii.gz",
                 atlasFile=atlasdir + "/" + "atlas*.nii")

filesource = Node(SelectFiles(templates), "filesource")
filesource.inputs.subject_id = "subj1"
filesource.outputs.get()

{'arimaFile': <undefined>, 'atlasFile': <undefined>}

In [31]:
# lets select region 1
# input is atlasFile, output is maskFile with region 1

def selectRegion(atlas_file):
    
    import numpy as np     
    import nibabel as nib     
    import os 
    
    atlas_img=nib.load(atlas_file)         
    atlas_array=np.asarray(atlas_img.dataobj).copy() # Avoid caching the proxy image
    
    n = 1
    region_array = atlas_array
    # binary mask the resulting image
    region_array[atlas_array <> n] = 0
    region_array[atlas_array == n] = 1
    
    region_image = nib.Nifti1Image(region_array, atlas_img.affine, atlas_img.header)
    
    result = os.getcwd() + '/' + 'region_image' + str(n) + '.nii'
    nib.save(region_image, result)
    return result
    



In [32]:
# node to compute a mask of region 1 of user atlas
computeRegion = Node(Function(input_names=['atlas_file', 'n_string'],
                                output_names=['out_file'],
                                function=selectRegion),
                                name='computeRegion')



connectivity.connect(filesource, "atlasFile", computeRegion, "atlas_file")


reg1 = selectRegion('/scratch/user/hirsch/datadir/data_set/t0009/repos01/Atlases/atlas_2reg.nii', 2)
print reg1

In [33]:
# lets calculate the mean residu signal eg. time courses in region 1


from nipype.interfaces.fsl.utils import ImageMeants

regMeants = Node(ImageMeants(), name="regMeants")     
regMeants.inputs.ignore_exception = False     
regMeants.inputs.order = 1     
regMeants.inputs.output_type = 'NIFTI_GZ'     
regMeants.inputs.terminal_output = 'stream'     
connectivity.connect(filesource, "arimaFile" , regMeants, "in_file")   
connectivity.connect(computeRegion, "out_file", regMeants, "mask")

In [34]:
# data sink
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/scratch/user/hirsch/datadir4/data_results_py'
connectivity.connect(regMeants,  'out_file', datasink, 'functionnal.regMeants')

In [35]:
connectivity.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node filesource in dir: /tmp/tmpUFNrNT/connectivity/filesource
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node computeRegion in dir: /tmp/tmp2aO_v4/connectivity/computeRegion
INFO:workflow:Executing node regMeants in dir: /tmp/tmpnuh3SU/connectivity/regMeants
INFO:workflow:Running: fslmeants -i /scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/arimaResidus/arima_residu.nii.gz -m /tmp/tmp2aO_v4/connectivity/computeRegion/region_image1.nii --order=1 -o /tmp/tmpnuh3SU/connectivity/regMeants/arima_residu_ts.txt
INFO:workflow:Executing node datasink in dir: /tmp/tmp1J1Mlj/connectivity/datasink
INFO:workflow:Runtime memory and threads stats unavailable


# fin du pipe